In [1]:
def normalize_text(text):
    punctuations = [".", ",", ":", ";" "!", "?"]

    normalized_text = text
    for punctuation in punctuations:
        normalized_text = normalized_text.replace(punctuation, "")

    normalized_text = normalized_text.lower()

    return normalized_text

In [2]:
def tokenize_text(text):
    return text.split()

In [3]:
def serialize_percentage_tokens(tokens):
    # Putting value and percentage symbol pairs into one token
    percentage_symbols = ["%", "százalék"]
    unused_tokens = []
    for i in range(len(tokens)):
        for percentage_symbol in percentage_symbols:
            if percentage_symbol == tokens[i]:
                tokens[i-1] = tokens[i-1]+"%"
                unused_tokens.append(tokens[i])
    for i in unused_tokens:
        tokens.remove(i)

In [4]:
def compare_two_tokens(token_a, token_b):
    best_match = 0.0
    offset = 0
    while(offset <= len(token_b) - len(token_a)):
        count = 0
        for i in range(len(token_a)):
            if(token_a[i] == token_b[offset + i]):
                count += 1
            else:
                break
        match_value = count/len(token_a)

        if(match_value > best_match):
            best_match = match_value
        offset += 1
    return best_match

In [5]:
def find_best_match_for_token_among_tokens(test_token, tokens):
    best_match = 0.0
    for token in tokens:
        match_value = compare_two_tokens(token, test_token)
        if(match_value > best_match):
            best_match = match_value
            #print(token, test_token, match_value)
    return best_match

In [6]:
def find_classes(tokens, synonyms):
    threshold = 0.8
    result_classes = []
    for i in range(len(tokens)):
        for key in synonyms.keys():
            match_value = find_best_match_for_token_among_tokens(tokens[i], synonyms[key])
            if(match_value >= threshold):
                found_class = {"token_id" : i, "class" : key}
                if not found_class in result_classes:
                    result_classes.append(found_class)
    return result_classes

In [7]:
def find_percentage_values(tokens):
    res = []
    for i in range(len(tokens)):
        if "%" in tokens[i]:
            prob = int(tokens[i].split("%")[0]) / 100
            found_class = {"token_id" : i, "value" : prob}
            if not found_class in res:
                res.append(found_class)
    return res

In [8]:
def make_json_from_classes_old(res_classes, res_percentages, class_names):
    # Calculating the weights according to the found classes
    base_data = dict()
    
    for class_entity in class_names:
        base_data[class_entity] = 0.0
    
    if(len(res_percentages) > 0):
        for i in range(len(res_percentages)):
            base_data[res_classes[i]['class']] = res_percentages[i]['value']
    else:
        for found_class in map(lambda x: x["class"], res_classes):
            base_data[found_class] = 1 / len(res_classes)
    
    return base_data

In [9]:
#TODO: ez nagyon csúnya, szépíteni kéne rajta...
# Pl az if-en belül lévő dolgokat kirobbantani onnan...

def make_json_from_classes(res_classes, res_probs, class_names):
    base_data = dict()

    for class_entity in class_names:
        base_data[class_entity] = 0.0

    if(len(res_probs) > 0):
        found_classes = []
        sums_of_prod = 0
        for i in range(len(res_probs)):
            for j in range(len(res_classes)):
                if(res_probs[i]["token_id"] < res_classes[j]["token_id"]):
                    base_data[res_classes[j]['class']] = res_probs[i]['value']
                    sums_of_prod += res_probs[i]['value']
                    found_classes.append(res_classes[j])
                    break
        if(sums_of_prod != 1):
            if(len(found_classes) < len(res_classes)):
                for found_class in found_classes:
                    res_classes.remove(found_class)
                for res_class in res_classes:
                    base_data[res_class['class']] = (1 - sums_of_prod) / len(res_classes)
            elif(len(found_classes) == len(res_classes)):
                remaining_classes = list(class_names)
                for found_class in found_classes:
                    remaining_classes.remove(found_class['class'])
                for remaining_class in remaining_classes:
                    base_data[remaining_class] = (1 - sums_of_prod) / len(remaining_classes)
    
    else:
        for found_class in map(lambda x: x["class"], res_classes):
            base_data[found_class] = 1 / len(res_classes)
    return base_data

In [10]:
def make_structured_data(input_sentence, synonyms):

    normalized_text = normalize_text(input_sentence)
    tokens = tokenize_text(normalized_text)
    serialize_percentage_tokens(tokens)
    
    result_classes = find_classes(tokens, synonyms)
    result_percentages = find_percentage_values(tokens)
    
    return make_json_from_classes(result_classes, result_percentages, synonyms.keys())

In [11]:
# TODO: ezeket file-ban tárolni, mint szótár az adott modellhez...

# from https://szinonimaszotar.hu
synonyms_afhq = {
    "cat": ["macska", "cica", "cicus", "cirmos", "cicamica", "kandúr", "macsi", "macsek", "cicó", "cila", "macs", "cilamila", "macskusz", "cicuska", "ciculi", "cirmi", "cicuka", "cicmic", "cic", "ciccancs", "kismacska", "mafka", "kiscica"],
    "dog": ["kutya", "eb", "kutyus", "kutyuli", "blöki", "véreb", "öleb", "kutyuska", "csahos", "házőrző", "kutyi", "bolhazsák", "kutyi-mutyi", "kutyuli-mutyuli", "vahúr", "négylábú", "kutyu", "kutyóka"],
    "wild": ["vadállat", "vad", "szörny", "szörnyeteg", "fenevad", "bestia", "dúvad", "fúria"]
}

synonyms_cifar = {
    "airplane": ["repülő", "repcsi", "göbzi", "repülőgép", "légibusz", "vasmadár", "légijármű"],
    "automobile": ["autó", "gépkocsi", "gépjármű", "személygépkocsi", "személygépjármű", "automobil", "jármű", "verda", "kocsi", "gép", "járgány", "tragacs", "négykerekű"],
    "bird": ["madár", "szárnyas", "csicsergő", "égivándor", "csirip", "csőrös"],
    "cat": ["macska", "cica", "cicus", "cirmos", "cicamica", "kandúr", "macsi", "macsek", "cicó", "cila", "macs", "cilamila", "macskusz", "cicuska", "ciculi", "cirmi", "cicuka", "cicmic", "cic", "ciccancs", "kismacska", "mafka", "kiscica"],
    "deer": ["szarvas", "agancsos"],
    "dog": ["kutya", "eb", "kutyus", "kutyuli", "blöki", "véreb", "öleb", "kutyuska", "csahos", "házőrző", "kutyi", "bolhazsák", "kutyi-mutyi", "kutyuli-mutyuli", "vahúr", "négylábú", "kutyu", "kutyóka"],
    "frog": ["béka", "breki", "varangy", "brekkencs"],
    "horse": ["ló", "táltos", "mén", "csődör", "paci", "lovacska", "kanca", "hátasló", "paripa", "csataló", "csikó", "hátas", "cocó", "csida", "gebe", "pejkó", "patás"],
    "ship": ["hajó", "ladik", "csónak", "sajka", "naszád", "bárka", "gálya", "yacht", "vitorlás", "kajak", "kenu", "lélekvesztő", "vízi jármű", "evezős hajó"],
    "truck": ["kamion", "teherautó", "nyergesvontató", "tehergépkocsi"]
}

In [12]:
make_structured_data("75 % macska van a képen, 25 százalék pedig öleb.", synonyms_afhq)

{'cat': 0.75, 'dog': 0.25, 'wild': 0.0}

In [13]:
make_structured_data("Egy vadkutya portréja található a képen.", synonyms_afhq)

{'cat': 0.0, 'dog': 0.5, 'wild': 0.5}

In [14]:
make_structured_data("Egy vadmacskutyáról készült fotó.", synonyms_afhq)

{'cat': 0.3333333333333333,
 'dog': 0.3333333333333333,
 'wild': 0.3333333333333333}

In [15]:
make_structured_data("Egy hajó a tengeren.", synonyms_cifar)

{'airplane': 0.0,
 'automobile': 0.0,
 'bird': 0.0,
 'cat': 0.0,
 'deer': 0.0,
 'dog': 0.0,
 'frog': 0.0,
 'horse': 0.0,
 'ship': 1.0,
 'truck': 0.0}

In [16]:
make_structured_data("Fotó egy olyan lóról amely 30%-ban agancsos.", synonyms_cifar)

{'airplane': 0.0,
 'automobile': 0.0,
 'bird': 0.0,
 'cat': 0.0,
 'deer': 0.3,
 'dog': 0.0,
 'frog': 0.0,
 'horse': 0.7,
 'ship': 0.0,
 'truck': 0.0}

In [17]:
make_structured_data("A macskutya a cifar adathalmazban is kísért!", synonyms_cifar)

{'airplane': 0.0,
 'automobile': 0.0,
 'bird': 0.0,
 'cat': 0.5,
 'deer': 0.0,
 'dog': 0.5,
 'frog': 0.0,
 'horse': 0.0,
 'ship': 0.0,
 'truck': 0.0}

In [18]:
# Halló - ló
make_structured_data("70%-nyi Hallókészülék.", synonyms_cifar)

{'airplane': 0.03333333333333334,
 'automobile': 0.03333333333333334,
 'bird': 0.03333333333333334,
 'cat': 0.03333333333333334,
 'deer': 0.03333333333333334,
 'dog': 0.03333333333333334,
 'frog': 0.03333333333333334,
 'horse': 0.7,
 'ship': 0.03333333333333334,
 'truck': 0.03333333333333334}